In [1]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

In [3]:
# Configurar Chrome
options = Options()
#options.add_argument("--headless")  # Comenta esto para ver el navegador
#options.add_argument("--start-maximized")
options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome(options=options)

In [5]:
driver.get("https://steamcommunity.com/app/1476970/reviews/?browsefilter=toprated&snr=1_5_100010_&l=english")
print(driver.title)  # Verifica que cargó la página correctamente

Steam Community :: IdleOn - The Idle RPG


In [7]:
time.sleep(5)

# Scroll dinámico
max_scrolls = 100
prev_count = 0
reviews = []  # Declarar fuera del bucle

for i in range(max_scrolls):
    print(f"Haciendo scroll #{i+1}...")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(4)
    current_reviews = driver.find_elements(By.CLASS_NAME, "apphub_Card")
    if len(current_reviews) == prev_count:
        break
    prev_count = len(current_reviews)
    reviews = current_reviews

print(f"\nTotal de reseñas cargadas: {len(reviews)}\n")


Haciendo scroll #1...
Haciendo scroll #2...
Haciendo scroll #3...
Haciendo scroll #4...
Haciendo scroll #5...
Haciendo scroll #6...
Haciendo scroll #7...
Haciendo scroll #8...
Haciendo scroll #9...
Haciendo scroll #10...
Haciendo scroll #11...
Haciendo scroll #12...
Haciendo scroll #13...
Haciendo scroll #14...
Haciendo scroll #15...
Haciendo scroll #16...
Haciendo scroll #17...
Haciendo scroll #18...
Haciendo scroll #19...
Haciendo scroll #20...
Haciendo scroll #21...
Haciendo scroll #22...
Haciendo scroll #23...
Haciendo scroll #24...
Haciendo scroll #25...
Haciendo scroll #26...
Haciendo scroll #27...
Haciendo scroll #28...
Haciendo scroll #29...
Haciendo scroll #30...
Haciendo scroll #31...
Haciendo scroll #32...
Haciendo scroll #33...
Haciendo scroll #34...
Haciendo scroll #35...
Haciendo scroll #36...
Haciendo scroll #37...
Haciendo scroll #38...
Haciendo scroll #39...
Haciendo scroll #40...
Haciendo scroll #41...
Haciendo scroll #42...
Haciendo scroll #43...
Haciendo scroll #44.

In [9]:
print(reviews)

[<selenium.webdriver.remote.webelement.WebElement (session="29c127311bb28c0fe08645644ceb8634", element="f.A06B0E19AA7E66A6E192A405B96E94FE.d.95F95D57FE33E9223A2D402853FF9B0C.e.46")>, <selenium.webdriver.remote.webelement.WebElement (session="29c127311bb28c0fe08645644ceb8634", element="f.A06B0E19AA7E66A6E192A405B96E94FE.d.95F95D57FE33E9223A2D402853FF9B0C.e.47")>, <selenium.webdriver.remote.webelement.WebElement (session="29c127311bb28c0fe08645644ceb8634", element="f.A06B0E19AA7E66A6E192A405B96E94FE.d.95F95D57FE33E9223A2D402853FF9B0C.e.48")>, <selenium.webdriver.remote.webelement.WebElement (session="29c127311bb28c0fe08645644ceb8634", element="f.A06B0E19AA7E66A6E192A405B96E94FE.d.95F95D57FE33E9223A2D402853FF9B0C.e.49")>, <selenium.webdriver.remote.webelement.WebElement (session="29c127311bb28c0fe08645644ceb8634", element="f.A06B0E19AA7E66A6E192A405B96E94FE.d.95F95D57FE33E9223A2D402853FF9B0C.e.50")>, <selenium.webdriver.remote.webelement.WebElement (session="29c127311bb28c0fe08645644ceb86

In [9]:
with open("raw_reviews.csv", mode="w", encoding="utf-8", newline='') as file:
    writer = csv.writer(file, delimiter="|", quoting=csv.QUOTE_MINIMAL)
    writer.writerow(["id", "html_review"])

    for i, review in enumerate(reviews):
        html = review.get_attribute("innerHTML").strip()
        html = html.replace("|", "")
        writer.writerow([i+1, html])

print("HTML de reseñas guardado en raw_reviews.csv")


HTML de reseñas guardado en raw_reviews.csv


In [13]:
driver.quit()

### extraer del html a un dataframe => csv las columnas necesarias.

In [11]:
import pandas as pd
from bs4 import BeautifulSoup
import re

In [15]:
# Cargar CSV original
df = pd.read_csv("raw_reviews.csv", delimiter="|")


In [17]:
# Función para extraer información
def parse_html(html):
    soup = BeautifulSoup(html, "html.parser")

    # Clasificación: "Recommended" o "Not Recommended"
    title = soup.find("div", class_="title")
    classification = title.text.strip() if title else "Sin clasificar"

    # Horas jugadas
    hours_div = soup.find("div", class_="hours")
    hours = hours_div.text.strip() if hours_div else "Sin horas"

    # Comentario
    comment_div = soup.find("div", class_="apphub_CardTextContent")
    comment = comment_div.get_text(separator=" ").strip() if comment_div else "Comentario no encontrado"

    # Productos en cuenta
    products_div = soup.find("div", class_="apphub_CardContentMoreLink ellipsis") 
    productos_en_cuenta = None
    if products_div:
        match = re.search(r'(\d+)', products_div.text)
        productos_en_cuenta = int(match.group(1)) if match else None

    # Reacciones totales
    reactions_div = soup.find("div", class_="found_helpful")
    reacciones_totales = None
    if reactions_div:
        match = re.search(r'(\d+)', reactions_div.text.replace(",", ""))
        reacciones_totales = int(match.group(1)) if match else None

    # Fecha de publicación
    date_div = soup.find("div", class_="date_posted")
    fecha_de_publicacion = date_div.text.replace("Posted: ", "").strip() if date_div else None

    # Nombre de usuario
    user_div = soup.find("div", class_="apphub_CardContentAuthorName")
    username = user_div.text.strip() if user_div else "Usuario desconocido"

    return classification, hours, comment, productos_en_cuenta, reacciones_totales, fecha_de_publicacion, username


In [19]:

# Aplicar la función a cada fila
df[[ 
    "user_clasification",
    "user_hours",
    "user_comment",
    "productos_en_cuenta",
    "reacciones_totales",
    "fecha_de_publicacion",
    "user_nickname"
]] = df["html_review"].apply(lambda html: pd.Series(parse_html(html)))

# Mostrar algunas filas
print(df[[
    "id",
    "user_clasification",
    "user_hours",
    "user_comment",
    "productos_en_cuenta",
    "reacciones_totales",
    "fecha_de_publicacion",
    "user_nickname"
]].head())

# Guardar nuevo CSV limpio
df.to_csv("raw_reviews_processed.csv", index=False, sep="|", encoding="utf-8")

print("\n✅ ¡CSV limpio guardado como 'raw_reviews_processed.csv' con columnas nuevas!")

   id user_clasification              user_hours  \
0   1    Not Recommended   1,317.6 hrs on record   
1   2        Recommended     676.8 hrs on record   
2   3    Not Recommended  11,760.1 hrs on record   
3   4    Not Recommended     234.6 hrs on record   
4   5    Not Recommended      18.3 hrs on record   

                                        user_comment  productos_en_cuenta  \
0  Posted: 17 June, 2023 \n Early Access Review \...                285.0   
1  Posted: 1 July, 2023 \n Early Access Review \n...                134.0   
2  Posted: 26 June, 2023 \n Early Access Review \...                  NaN   
3  Posted: 29 April, 2024 \n Early Access Review ...                  NaN   
4  Posted: 17 June, 2023 \n Early Access Review \...                  7.0   

   reacciones_totales fecha_de_publicacion  user_nickname  
0                1606        17 June, 2023           Ivan  
1                 597         1 July, 2023       Mocamoca  
2                 537        26 June, 2023  